<a href="https://colab.research.google.com/github/Yashsharma009/new/blob/main/face-detectionmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install opencv-python-headless
!pip install cmake
!pip install dlib
!pip install face_recognition
!pip install IPython


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=b89ccf2105334724d57bf65a72c46c8b81f7eb0812047124a63dc360aeca92e9
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.3 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
import csv
import matplotlib.pyplot as plt

In [ ]:
train_dir = '/content/drive/MyDrive/train'
train_images = []
classNames = []
rollNumbers = []
classes = []
myList = os.listdir(train_dir)
for cl in myList:
    curImg = cv2.imread(os.path.join(train_dir, cl))
    train_images.append(curImg)
    parts = os.path.splitext(cl)[0].split('_')
    if len(parts) >= 3:
        name = '_'.join(parts[:-2])
        roll_number = parts[-2]
        class_name = parts[-1]
    else:
        name = parts[0]
        roll_number = ""
        class_name = ""
    classNames.append(name)
    rollNumbers.append(roll_number)
    classes.append(class_name)

print('Names:', classNames)
print('Roll Numbers:', rollNumbers)
print('Classes:', classes)

encodeListKnown = []
for img in train_images:
    if img is not None:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        face_encodings = face_recognition.face_encodings(img)
        if len(face_encodings) > 0:
            encodeListKnown.append(face_encodings[0])

print('Encoding Complete')

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
# Define the path to the input video
input_video_path = '/content/drive/MyDrive/20230715_215944.mp4'

# Open the video capture
video_capture = cv2.VideoCapture(input_video_path)

# Get the total number of frames and calculate the frames per second (fps)
total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(video_capture.get(cv2.CAP_PROP_FPS))

# Calculate the frames to skip per second
frames_to_skip = int(round(fps / 1))  # Process 1 frame per second

# Initialize an empty list to store the attendance data
attendance_data = []
processed_names = set()  # Set to store unique names

frame_count = 0

while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    frame_count += 1

    # Skip frames if necessary
    if frame_count % frames_to_skip != 0:
        continue

    input_image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    face_locations = face_recognition.face_locations(input_image_rgb, model="cnn")

    print(f"Number of Faces Detected: {len(face_locations)}")

    encodings = face_recognition.face_encodings(input_image_rgb, face_locations)

    for face_encoding, face_location in zip(encodings, face_locations):
        matches = face_recognition.compare_faces(encodeListKnown, face_encoding)
        face_distances = face_recognition.face_distance(encodeListKnown, face_encoding)

        if len(face_distances) > 0:
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = classNames[best_match_index]
                roll_number = rollNumbers[best_match_index]
                class_name = classes[best_match_index]
                if name in processed_names:
                    continue  # Skip writing the record if the name has already been processed

                processed_names.add(name)  # Add the name to the set of processed names

                attendance_data.append([name, roll_number, class_name])

                top, right, bottom, left = face_location
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.putText(frame, f'{name} ({roll_number}, {class_name})', (left + 6, bottom - 6),
                            cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            else:
                # Keep the non-matching faces blank
                cv2.rectangle(frame, (face_location[3], face_location[0]), (face_location[1], face_location[2]),
                              (0, 0, 0), cv2.FILLED)

    # Display the frame with detections
    cv2_imshow(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close OpenCV windows
video_capture.release()
cv2.destroyAllWindows()

attendance_file = 'Attendance.csv'

with open(attendance_file, 'w', newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(['Name', 'Roll Number', 'Class', 'Timestamp'])
    for data in attendance_data:
        name, roll_number, class_name = data
        now = datetime.now()
        dtString = now.strftime('%H:%M:%S')
        csv_writer.writerow([name, roll_number, class_name, dtString])

# Read the CSV file and display its contents
with open(attendance_file, 'r') as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        print(row)
